In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# Load data_weekly files
data_folder = Path("data")

data_weekly_A = pd.read_csv(data_folder / 'data_weekly_A.csv', index_col=0, parse_dates=True)
data_weekly_B = pd.read_csv(data_folder / 'data_weekly_B.csv', index_col=0, parse_dates=True)
data_weekly_C = pd.read_csv(data_folder / 'data_weekly_C.csv', index_col=0, parse_dates=True)
data_weekly_D = pd.read_csv(data_folder / 'data_weekly_D.csv', index_col=0, parse_dates=True)
data_weekly_E = pd.read_csv(data_folder / 'data_weekly_E.csv', index_col=0, parse_dates=True)

print("Data loaded successfully!")
print(f"data_weekly_A: {data_weekly_A.shape}")
print(f"data_weekly_B: {data_weekly_B.shape}")
print(f"data_weekly_C: {data_weekly_C.shape}")
print(f"data_weekly_D: {data_weekly_D.shape}")
print(f"data_weekly_E: {data_weekly_E.shape}")